<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e6048e35e7f60874c9fc8c1d0cfc73681f7af33357de96a1d8278e010aed1911
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-13 15:05:32
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: -97.32 K (-0.69%)
Current PnL: -21.23 L (-14.15%)
CY Booked + Current PnL: -9.33 L (-6.22%)
-------------------
Total profit:  1.23 L
Total loss:  -22.46 L
-------------------
Total Booked + Current PnL: 17.65 L (14.33%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.48%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.44 L (63.06%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,58.0,X-LC,3.69,214035.0,9535.0,13762.0,0.03,4.66,6.43,11.39,10.0,0.69,1.54,14.03,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,93.81,41.0,M-SC,1.25,83512.0,-17265.0,17354.0,0.35,-17.13,20.78,0.09,245.0,-0.99,0.60,9.72,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-1.64,50.0,X-MC,1.04,206475.0,14733.0,23022.0,-2.88,7.68,11.15,19.69,99.0,0.64,1.48,15.00,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,47.0,X-LC,2.97,214375.0,4009.0,24782.0,-0.54,1.91,11.56,13.68,37.0,0.16,1.54,18.74,XY25,NTT,REFINERIES
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195282.0,-4856.0,25738.0,-0.86,-2.43,13.18,10.44,4.0,-0.19,1.40,3.00,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIS,528.00,2029.02,44.0,H-SC,3.78,85922.0,-25110.0,48718.0,3.60,-22.62,56.70,21.26,156.0,-0.52,0.62,16.09,OX40N,NTT,MISC
45,KPIGREEN,731.05,-4.41,46.0,H-SC,6.23,113841.0,-11456.0,70388.0,2.07,-9.14,61.83,47.03,141.0,-0.16,0.82,42.54,MH,ATH,POWER
23,DEN,75.00,96.89,47.0,M-SC,3.20,55424.0,-29040.0,64575.0,1.85,-34.38,116.51,42.07,237.0,-0.45,0.40,17.70,AR,NTT,ENTERTAINMENT
15,CAMPUS,393.00,-22.11,65.0,M-SC,2.38,162279.0,-5496.0,61341.0,1.64,-3.28,37.80,33.28,210.0,-0.09,1.17,25.78,XY24,NTT,FOOTWEAR
63,SHALBY,327.00,1199.84,59.0,M-SC,22.43,176715.0,-4428.0,49904.0,1.17,-2.44,28.24,25.10,235.0,-0.09,1.27,41.62,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GREENPANEL,537.00,223.00,52.0,M-SC,4.41,144302.0,-38776.0,117750.0,-4.55,-21.18,81.60,43.14,230.0,-0.33,1.04,35.74,XY24,NTT,MISC
20,COFFEEDAY,80.00,-43.66,50.0,L-SC,29.06,81984.0,-31565.0,71613.0,-3.20,-27.80,87.35,35.27,268.0,-0.44,0.59,100.66,XR,NTT,HOTELS
53,RAJOOENG,143.10,-40.86,46.0,H-SC,16.77,89740.0,-12760.0,53359.0,-3.14,-12.45,59.46,39.61,114.0,-0.24,0.64,9.60,AR,ATH,MISC
14,BSOFT,836.99,-20.99,36.0,H-SC,14.01,88791.0,-51862.0,124645.0,-3.13,-36.87,140.38,51.74,131.0,-0.42,0.64,1.25,XR,ATH,IT
46,LAOPALA,464.00,100.59,35.0,H-SC,1.20,91734.0,-39371.0,90615.0,-3.03,-30.03,98.78,39.09,142.0,-0.43,0.66,18.15,AR,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.0,45.36,44.0,M-LC,6.33,204105.0,935.0,41188.0,-0.50,0.46,20.18,20.74,55.0,0.02,1.47,6.03,XY25,NTT,FINANCE
85,WIPRO,420.0,-13.34,49.0,M-LC,5.83,152024.0,1079.0,108378.0,-1.41,0.71,71.29,72.51,53.0,0.01,1.09,6.78,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,64.0,H-SC,11.33,131625.0,-10854.0,30379.0,-0.06,-7.62,23.08,13.70,163.0,-0.36,0.95,55.35,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,93.81,41.0,M-SC,1.25,83512.0,-17265.0,17354.0,0.35,-17.13,20.78,0.09,245.0,-0.99,0.60,9.72,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,50.0,H-SC,1.98,221724.0,-47943.0,84277.0,-0.46,-17.78,38.01,13.47,138.0,-0.57,1.59,12.74,XY24,NTT,PAINTS
17,CERA,9475.0,-24.34,36.0,H-SC,1.90,138012.0,-37891.0,79909.0,0.56,-21.54,57.90,23.89,149.0,-0.47,0.99,19.19,OX40N,NTT,CERAMICS
65,SIS,528.0,2029.02,44.0,H-SC,3.78,85922.0,-25110.0,48718.0,3.60,-22.62,56.70,21.26,156.0,-0.52,0.62,16.09,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,44.0,M-LC,6.33,204105.0,935.0,41188.0,-0.50,0.46,20.18,20.74,55.0,0.02,1.47,6.03,XY25,NTT,FINANCE
59,SAIL,228.00,45.15,51.0,M-MC,12.66,228509.0,3547.0,164115.0,0.40,1.58,71.82,74.53,192.0,0.02,1.64,34.34,XY24,BTT,STEEL
31,HINDZINC,730.22,34.27,66.0,M-LC,8.30,219240.0,14164.0,98395.0,1.14,6.91,44.88,54.89,52.0,0.14,1.58,32.67,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,58.0,M-LC,9.39,158998.0,18158.0,26330.0,0.41,12.89,16.56,31.59,66.0,0.69,1.14,42.80,XY24,NTT,BANKS
85,WIPRO,420.00,-13.34,49.0,M-LC,5.83,152024.0,1079.0,108378.0,-1.41,0.71,71.29,72.51,53.0,0.01,1.09,6.78,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-24.22,68.0,H-MC,12.35,195188.0,15380.0,97028.0,0.11,8.55,49.71,62.52,88.0,0.16,1.40,45.56,XR,NTT,BANKS
31,HINDZINC,730.22,34.27,66.0,M-LC,8.30,219240.0,14164.0,98395.0,1.14,6.91,44.88,54.89,52.0,0.14,1.58,32.67,X5K,ATH,METALS
36,INDIAMART,4810.62,-54.45,43.0,H-SC,8.51,127041.0,3705.0,127918.0,0.23,3.00,100.69,106.72,119.0,0.03,0.91,27.08,AR,ATH,MISC
85,WIPRO,420.00,-13.34,49.0,M-LC,5.83,152024.0,1079.0,108378.0,-1.41,0.71,71.29,72.51,53.0,0.01,1.09,6.78,XR,NTT,IT
84,WHIRLPOOL,2270.00,-48.24,32.0,M-SC,2.90,87518.0,-3980.0,82731.0,-0.22,-4.35,94.53,86.07,223.0,-0.05,0.63,27.09,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,32.0,X-SC,37.78,52122.0,-12884.0,163814.0,-2.96,-19.82,314.29,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.31,33.0,X-MC,0.90,229788.0,-17943.0,92995.0,-1.53,-7.24,40.47,30.30,92.0,-0.19,1.65,5.09,X40,ATH,ELECTRICAL
9,BAJAJHFL,181.50,-15.33,34.0,X-MC,5.15,178383.0,-22123.0,119285.0,-1.12,-11.03,66.87,48.45,90.0,-0.19,1.28,2.87,X40N,ATH,FINANCE
22,DABUR,735.00,-12.66,35.0,X-MC,3.57,236438.0,-11630.0,120040.0,-0.45,-4.69,50.77,43.70,102.0,-0.10,1.70,9.08,XY24,BTT,FMCG
5,ASIANPAINT,3465.66,-18.16,35.0,X-LC,5.47,206439.0,-45329.0,98533.0,0.24,-18.00,47.73,21.13,27.0,-0.46,1.48,11.04,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,HAVELLS,2069.16,-7.31,33.0,X-MC,0.90,229788.0,-17943.0,92995.0,-1.53,-7.24,40.47,30.30,92.0,-0.19,1.65,5.09,X40,ATH,ELECTRICAL
83,VOLTAS,1530.00,-1.64,50.0,X-MC,1.04,206475.0,14733.0,23022.0,-2.88,7.68,11.15,19.69,99.0,0.64,1.48,15.00,XY25,NTT,AC
72,TATAMOTORS,1065.00,-24.14,37.0,X-LC,1.17,239326.0,-37197.0,147257.0,-2.89,-13.45,61.53,39.81,54.0,-0.25,1.72,14.70,XY24,NTT,AUTO
64,SIEMENS,4671.50,-4.32,40.0,H-LC,1.28,153785.0,-32310.0,79784.0,-2.28,-17.36,51.88,25.51,15.0,-0.40,1.11,13.33,AR,ATH,ELECTRICAL
33,ICICIGI,2260.25,-21.27,37.0,X-MC,1.64,135575.0,-558.0,31684.0,-1.65,-0.41,23.37,22.86,91.0,-0.02,0.97,12.15,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,32.0,X-SC,37.78,52122.0,-12884.0,163814.0,-2.96,-19.82,314.29,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
82,VBL,671.64,-21.55,38.0,X-LC,10.47,282283.0,-33559.0,146223.0,-0.27,-10.63,51.80,35.67,5.0,-0.23,2.03,1.76,X40N,ATH,FMCG
9,BAJAJHFL,181.50,-15.33,34.0,X-MC,5.15,178383.0,-22123.0,119285.0,-1.12,-11.03,66.87,48.45,90.0,-0.19,1.28,2.87,X40N,ATH,FINANCE
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195282.0,-4856.0,25738.0,-0.86,-2.43,13.18,10.44,4.0,-0.19,1.40,3.00,X40,NTT,FMCG
21,COLPAL,3767.14,-4.76,39.0,X-MC,7.44,222210.0,-41155.0,154503.0,-0.26,-15.63,69.53,43.04,84.0,-0.27,1.60,3.16,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,32.0,X-SC,37.78,52122.0,-12884.0,163814.0,-2.96,-19.82,314.29,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-40.56,36.0,X-SC,2.88,81009.0,-64151.0,140081.0,-0.85,-44.19,172.92,52.31,136.0,-0.46,0.58,8.44,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.15,38.0,X-SC,4.41,95392.0,-33278.0,78574.0,-1.69,-25.86,82.37,35.20,219.0,-0.42,0.69,9.49,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-4.94,51.0,X-SC,1.45,108102.0,6098.0,37825.0,0.77,5.98,34.99,43.06,122.0,0.16,0.78,24.12,X40N,ATH,MISC
32,HONAUT,58357.33,-23.00,45.0,X-SC,2.21,108495.0,-19443.0,66573.0,-0.85,-15.20,61.36,36.84,143.0,-0.29,0.78,11.07,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-29.01,48.0,X-LC,12.19,282639.0,-63317.0,131568.0,-0.71,-18.30,46.55,19.73,1.0,-0.48,2.03,4.10,X40,ATH,IT
39,INFY,2275.00,-18.16,53.0,X-LC,7.63,318605.0,5717.0,165961.0,-1.26,1.83,52.09,54.87,3.0,0.03,2.29,8.60,X40,BTT,IT
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195282.0,-4856.0,25738.0,-0.86,-2.43,13.18,10.44,4.0,-0.19,1.40,3.00,X40,NTT,FMCG
82,VBL,671.64,-21.55,38.0,X-LC,10.47,282283.0,-33559.0,146223.0,-0.27,-10.63,51.80,35.67,5.0,-0.23,2.03,1.76,X40N,ATH,FMCG
1,ABB,7934.00,-41.03,42.0,H-LC,7.43,246528.0,-15091.0,134308.0,-0.96,-5.77,54.48,45.57,7.0,-0.11,1.77,4.90,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-43.66,50.0,L-SC,29.06,81984.0,-31565.0,71613.0,-3.20,-27.80,87.35,35.27,268.0,-0.44,0.59,100.66,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,42.0,H-SC,10.04,90555.0,-7425.0,29032.0,-0.97,-7.58,32.06,22.05,152.0,-0.26,0.65,31.63,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,108.87,64.0,H-SC,11.33,131625.0,-10854.0,30379.0,-0.06,-7.62,23.08,13.70,163.0,-0.36,0.95,55.35,OX40N,NTT,BANKS
60,SAMMAANCAP,326.00,-184.09,65.0,M-SC,16.40,146088.0,-3132.0,147315.0,0.06,-2.10,100.84,96.62,208.0,-0.02,1.05,58.10,XY25,NTT,FINANCE
68,SURYODAY,240.00,64.47,61.0,H-SC,9.87,150603.0,-28468.0,94202.0,-2.23,-15.90,62.55,36.71,135.0,-0.30,1.08,49.43,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-184.09,65.0,M-SC,16.40,146088.0,-3132.0,147315.0,0.06,-2.10,100.84,96.62,208.0,-0.02,1.05,58.10,XY25,NTT,FINANCE
68,SURYODAY,240.00,64.47,61.0,H-SC,9.87,150603.0,-28468.0,94202.0,-2.23,-15.90,62.55,36.71,135.0,-0.30,1.08,49.43,XR,NTT,BANKS
63,SHALBY,327.00,1199.84,59.0,M-SC,22.43,176715.0,-4428.0,49904.0,1.17,-2.44,28.24,25.10,235.0,-0.09,1.27,41.62,XY24,NTT,HEALTHCARE
31,HINDZINC,730.22,34.27,66.0,M-LC,8.30,219240.0,14164.0,98395.0,1.14,6.91,44.88,54.89,52.0,0.14,1.58,32.67,X5K,ATH,METALS
67,STARHEALTH,761.00,16.85,68.0,H-SC,7.84,260386.0,-20737.0,153602.0,-0.02,-7.38,58.99,47.26,144.0,-0.14,1.87,39.51,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.31
1,25,43.86
2,50,75.88


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.31
LC    33.99
MC    21.66
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.27
X40      14.69
XY25     11.94
XR       10.97
X40N      9.98
AR        8.17
OX40N     7.64
X200      1.78
X5K       1.58
SR        1.12
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.63
X-LC    23.04
X-MC    15.92
M-SC    12.68
M-LC     5.28
H-LC     4.66
X-SC     4.56
H-MC     3.79
M-MC     1.64
L-SC     1.44
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.90,-6.19,41.18
IT,12.93,-16.87,80.18
FINANCE,12.02,-12.89,64.06
MISC,6.95,-18.68,84.44
BANKS,6.50,-10.18,68.15
PAINTS,5.56,-19.37,37.16
ELECTRICAL,5.31,-11.48,50.59
INSURANCE,3.80,-4.36,43.37
AUTO,3.47,-18.96,75.83


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3218892.0,22
XR,1380693.0,14
AR,1192679.0,9
X40,794894.0,10
X40N,639543.0,8
OX40N,583698.0,10
XY25,521292.0,8
SR,265522.0,2
X5K,98395.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3428901.0,25
M-SC,1278423.0,15
X-MC,1110898.0,11
X-LC,1034679.0,13
X-SC,546254.0,6
H-MC,352002.0,3
H-LC,291825.0,3
M-LC,274291.0,4
L-SC,259955.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1193062.0      6
           AR         814243.0      5
           XR         813756.0      7
M-SC       XY24       685029.0      6
X-MC       XY24       588950.0      4
X-LC       X40        491182.0      5
X-SC       X40N       303866.0      4
H-SC       OX40N      271930.0      4
           SR         265522.0      2
X-MC       X40        225138.0      4
X-LC       X40N       216392.0      3
           XY24       214989.0      2
H-LC       AR         214092.0      2
H-MC       XY24       182603.0      1
X-MC       XY25       177525.0      2
L-SC       XR         170585.0      2
M-SC       AR         164344.0      2
M-MC       XY24       164115.0      1
X-SC       XY24       163814.0      1
M-SC       OX40N      150027.0      4
           XY25       147315.0      1
           XR         131708.0      2
X-MC       X40N       119285.0      1
X-LC       XY25       112116.0      3
M-LC       XR         108378.0      1
           X5K         98395.0      1
H-MC       XR          97028.0      1
L-SC       OX40N       89370.0      1
X-SC       X40         78574.0      1
H-LC       X200        77733.0      1
H-MC       OX40N       72371.0      1
H-SC       MH          70388.0      1
L-MC       XR          59238.0      1
L-LC       XY25        43148.0      1
M-LC       XY25        41188.0      1
           XY24        26330.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 47.0 seconds
